In [49]:
import numpy as np
import numpy.typing as npt
from matplotlib import pyplot as plt
from scipy.special import erf

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

plt.rcParams['figure.facecolor'] = 'white'

def EFP(time: np.int32 | npt.NDArray[np.int32], amplitude: np.float64, mu: np.float64, sigma: np.float64, tau: np.float64) -> np.float64 | npt.NDArray[np.float64]:
    Aprime = amplitude * sigma / (tau * np.sqrt(2 / np.pi))
    y1 = -(time - mu) / tau + sigma**2 / (2 * tau**2)
    y2 = (time - mu) / (np.sqrt(2) * sigma) - sigma / (np.sqrt(2) * tau)
    return Aprime * np.exp(y1) * (1 + erf(y2))

def plot_flare(identifier: str):
    dirname = ''
    datafile = f'{dirname}flares/{identifier}.csv'
    df = pd.read_csv(datafile)
    summaryfile = f'{dirname}allflares_preprocessed.csv'
    allcsv = pd.read_csv(summaryfile)
    row = allcsv[allcsv['identifier'] == identifier].iloc[0]

    fig, axs = plt.subplots(2, figsize=(9.6, 9.6), gridspec_kw={'height_ratios': [3, 1]})

    axs[0].set_title(f'{identifier}')
    plt.xlabel('Time (s)')
    plt.subplots_adjust(hspace=.0)
    linestyle = {"elinewidth":1, "capsize":0, "ecolor":"grey"}
    plt.setp(axs[0].get_xticklabels(), visible=False)

    xt = df['Time']
    yt = df['Counts']
    yerr = df['Error']

    axs[0].errorbar(xt, yt, yerr=yerr, fmt='D', ms=3, c='blue', **linestyle)

    for ax in axs:
        ax.tick_params(which='major', width=1)
        ax.tick_params(which='major', length=4)
        ax.margins(x=0)
    axs[0].set_ylabel('Flux (nW/m$^2$)')
    axs[1].set_ylabel('Residual ((o-f)/$\\sigma$)')
    axs[1].set_ylim(bottom=-4,top=4)

    range = (np.max(yt) - np.min(yt))/10
    axs[0].scatter(row['start_time'], \
        row['start_count'], c='k', s=30, zorder=15)
    axs[0].vlines(x=row['start_time'], \
        ymin=row['start_count'] - range, \
        ymax=row['end_count'] + range, color='k', linestyle='dotted', linewidth=2)
    axs[0].scatter(row['end_time'], \
        row['end_count'], c='k', s=30, zorder=15)
    axs[0].vlines(x=row['end_time'], \
        ymin=row['end_count'] - range, \
        ymax=row['end_count'] + range, color='k', linestyle='dotted', linewidth=2)

    yfit = EFP(xt, row['amplitude'], row['mu'], row['sigma'], row['tau'])
    axs[0].plot(xt, yfit, c='r', zorder=10, lw=3)
    axs[0].tick_params(axis='x', direction='in')
    axs[1].errorbar(xt, (yt-yfit)/np.std(yt-yfit), yerr=yerr/np.std(yt-yfit), fmt='D', ms=3, c='blue', **linestyle)
    axs[1].axhline(np.mean((yt-yfit)/np.std(yt-yfit)), linestyle='--', lw=1, c='navy')

    plt.show()

In [50]:
import pandas as pd
from scipy.signal import welch
from tqdm import tqdm

class_info = pd.read_csv('allflares_preprocessed.csv')

print(class_info)

data = []
labels = []
ids = []
peak_count = []

tot_flares = 0

for i, iden in enumerate(tqdm(np.array(class_info['identifier']))):
    if class_info['flare_type'][i] != 'A' and class_info['flare_type'][i] != 'B':
        continue

    try:
        tod = pd.read_csv('flares/' + iden + '.csv')
    except FileNotFoundError:
        continue

    if np.max(np.diff(tod['Time'])) > 10 or np.max(np.diff(tod['Time'])) < 10:
        continue

    tot_flares += 1
    data.append(list(tod['Counts']))
    peak_count.append(class_info['peak_count'][i])
    
    if class_info['flare_type'][i] == 'A':
        labels.append(0)
    else:
        labels.append(1)

    ids.append(class_info['identifier'][i])

    """
    print(np.array(tod['Counts']))
    
    plt.figure(figsize=(12, 8))

    plt.scatter(np.array(tod['Time']) - np.array(tod['Time'])[0], np.array(tod['Counts']))
    plt.errorbar(np.array(tod['Time']) - np.array(tod['Time'])[0], np.array(tod['Counts']), np.array(tod['Error']), ls='')

    plt.title(class_info['identifier'][i] + ': ' + class_info['flare_type'][i], fontsize=24)

    plt.show()
    """

labels = np.array(labels)

print(tot_flares)

          date identifier  start_time  start_count  peak_time  peak_count  \
0     20221104  20221104q     82785.0         9.56    82965.0      101.03   
1     20221104  20221104a      8135.0         4.08     8455.0       42.79   
2     20221104  20221104p     80175.0        49.67    80575.0      507.60   
3     20221104  20221104o     72505.0        10.25    73475.0      103.02   
4     20221104  20221104n     71875.0         0.00    74915.0       20.66   
...        ...        ...         ...          ...        ...         ...   
8039  20190921  20190921a     15395.0         0.32    15585.0        3.26   
8040  20190917  20190917b     70935.0         0.90    71015.0        7.00   
8041  20190917  20190917a     60615.0         0.25    60895.0        2.45   
8042  20190915  20190915a     65555.0         0.14    66025.0        1.37   
8043  20190914  20190914a     74075.0         0.19    74215.0        2.05   

      end_time  end_count  scpeaks0-05  multi_flare_region_flag  ...  \
0  

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8044/8044 [00:04<00:00, 2002.68it/s]

3704


In [51]:
data = pad_sequences(data, padding='post', dtype=float)

print(data[0])
print(data.shape)
print(type(data[0, 0]))

#print(labels)

#for d in data:
#    plt.scatter(range(len(d)), d)
#    
#    plt.show()

[15.6400759  13.31584982 25.55306329 ...  0.          0.
  0.        ]
(3704, 2089)
<class 'numpy.float64'>


In [52]:
model = Sequential()
    
# First convolutional layer
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(len(data[0]), 1)))
model.add(MaxPooling1D(pool_size=2))

# Second convolutional layer
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Third convolutional layer
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

/home/mrchu/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [53]:
model.fit(data, labels, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.7305 - loss: 4.3398 - val_accuracy: 0.8273 - val_loss: 0.4977
Epoch 2/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8030 - loss: 0.5291 - val_accuracy: 0.8273 - val_loss: 0.4704
Epoch 3/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7963 - loss: 0.5082 - val_accuracy: 0.8273 - val_loss: 0.4543
Epoch 4/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7897 - loss: 0.4975 - val_accuracy: 0.8273 - val_loss: 0.4625
Epoch 5/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8024 - loss: 0.4742 - val_accuracy: 0.8273 - val_loss: 0.4854
Epoch 6/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8114 - loss: 0.4545 - val_accuracy: 0.8259 - val_loss: 0.4540
Epoch 7/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8155 - loss: 0.4569 - val_accuracy: 0.8273 - val_loss: 0.4483
Epoch 8/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8038 - loss: 0.4506 - val_accuracy: 0.8300 - 

In [54]:
print(np.expand_dims(data, axis=-1).shape)

preds = model.predict(np.expand_dims(data, axis=-1))

print(preds)

(3704, 2089, 1)
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[9.9622744e-01]
 [1.9336412e-07]
 [1.0000000e+00]
 ...
 [9.9951065e-01]
 [8.2865191e-01]
 [9.9999917e-01]]


In [58]:
class_list = {'0': 'A', '1': 'B'}

num_high = 0
num_mis_high = 0
num_mis_low = 0

tot_exc = 0

for i, pred in enumerate(tqdm(preds)):
    conf = np.abs(pred[0]-0.5)+0.5
    pred_class = int(np.round(pred[0]))

    #plt.scatter(np.arange(0, len(data[i][data[i] != 0]))*10, data[i][data[i] != 0])

    #plt.xlabel('Time')
    #plt.ylabel('Count')

    #plt.title(f"{ids[i]}, Pred. Class: {class_list[str(pred_class)]}, Actual Class: {class_list[str(labels[i])]}, Conf = {np.round(conf, 3)}")

    if peak_count[i] < 10:
        #plt.close()
        continue

    tot_exc += 1
    
    if conf > 0.9:
        #plt.savefig(f'classifications/high_confidence/{ids[i]}.png')
        num_high += 1

        if class_list[str(pred_class)] != class_list[str(labels[i])]:
            num_mis_high += 1
            #plt.show()
    else:
        #plt.savefig(f'classifications/low_confidence/{ids[i]}.png')

        if class_list[str(pred_class)] != class_list[str(labels[i])]:
            num_mis_low += 1
            #plt.show()

    #plt.close()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3704/3704 [00:00<00:00, 122831.29it/s]


In [34]:
# cutoff = 0.8

print(f'Number of high confidence classifications = {num_high}')
print(f'Number of low confidence classifications = {len(preds) - num_high}')
print(f'Percentage of high confidence = {num_high/len(preds)}')

print(f'Total misclassified = {num_mis_high + num_mis_low}')
print(f'Accuracy in total = {1-(num_mis_high + num_mis_low)/len(preds)}')

print(f'Accuracy of high confidence = {1-num_mis_high/num_high}')
print(f'Accuracy of low confidence = {1-num_mis_low/(len(preds) - num_high)}')

Number of high confidence classifications = 3637
Number of low confidence classifications = 67
Percentage of high confidence = 0.9819114470842333
Total misclassified = 160
Accuracy in total = 0.9568034557235421
Accuracy of high confidence = 0.963706351388507
Accuracy of low confidence = 0.582089552238806


In [60]:
# cutoff = 0.9

print(f'tot = {tot_exc}')

print(f'Number of high confidence classifications = {num_high}')
print(f'Number of low confidence classifications = {tot_exc - num_high}')
print(f'Percentage of high confidence = {num_high/tot_exc}')

print(f'Total misclassified = {num_mis_high + num_mis_low}')
print(f'Accuracy in total = {1-(num_mis_high + num_mis_low)/tot_exc}')

print(f'Accuracy of high confidence = {1-num_mis_high/num_high}')
print(f'Accuracy of low confidence = {1-num_mis_low/(tot_exc - num_high)}')

tot = 3159
Number of high confidence classifications = 3071
Number of low confidence classifications = 88
Percentage of high confidence = 0.9721430832541944
Total misclassified = 85
Accuracy in total = 0.9730927508705286
Accuracy of high confidence = 0.982090524259199
Accuracy of low confidence = 0.6590909090909092
